# Generate features from emails to improve classification

- Raw email text data can be found at: https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tgz
- A breakdown of emails by sender can be found [here](data/emails_by_address/)

### Load minimal project dataset as `data_dict`

In [32]:
import os
import pickle
import re
import numpy as np

HOME_PATH = os.path.expanduser('~')
DATA_PATH = os.path.join(HOME_PATH, 'Desktop', 'raw_data', 'ml')

with open('data/final_project_dataset.pkl', 'r') as data_file:
    data_dict = pickle.load(data_file)

del data_dict['TOTAL']
del data_dict['LOCKHART EUGENE E']

I will focus on two sets of emails:
- Those already in the project dataset as well as in the email database (86 emails), 
- Those known to belong to POIs but not in the project dataset (an additional 4)

Make an `all_emails` list that contains both of these types

In [22]:
from tools.poi_email_addresses import poiEmails

#list of emails already in the dataset
data_emails = [(item['email_address'], item['poi']) for item in data_dict.values()]
data_emails = [tup for tup in data_emails if tup[0] != 'NaN']
print 'Minimal data emails: ', len(data_emails) 

#additional emails known to belong to pois
file_emails = poiEmails()
file_emails = [(email, True) for email in file_emails]
print 'Additional emails: ', len(file_emails)

#all together
all_emails = list(set(data_emails + file_emails))
print 'All together (removing duplicates): ', len(all_emails)
all_emails[183]

Minimal data emails:  111
Additional emails:  90
All together (removing duplicates):  184


('jskilling@enron.com', True)

Generate a list of paths for all of the emails sent and received by the emails listed in `all_emails`

In [20]:
def find_emailpaths(emails):
    """
    Given a list of emails generates tuples with the path to all the messages associated
    with an email.
    
    Input:
    emails determines what emails to include in output.
    Should be list of tuples with:
    tup[0] = email
    tup[1] = True if email belongs to POI False otherwise
    
    Output:
    List of tuples with:
    tup[0] = path to file with paths organized by email address
    tup[1] = email address
    tup[2] = whether emails were to or from email address
    tup[3] = True if email belongs to POI False otherwise
    """
    
    current_path = os.path.join(os.getcwd(), 'data', 'emails_by_address')
    all_filenames = os.listdir(current_path)
    emailpath_tuples = []
    
    emails, poi_vals = zip(*emails)

    for filename in all_filenames:
        #generate full path to file
        temp_path = (os.path.join(current_path, filename))
        
        #isolate email from filename and determine if to or from
        if 'from_' in filename:
            temp_filename = filename.replace('from_', '').replace('.txt', '')
            to_from = 'from'
        elif 'to_' in filename:
            temp_filename = filename.replace('to_', '').replace('.txt', '')
            to_from = 'to'
        
        #only include emails from 
        if temp_filename in emails:
            i = emails.index(temp_filename)
            tup = (temp_path, temp_filename, to_from, poi_vals[i])
            emailpath_tuples.append(tup)

    return emailpath_tuples

In [23]:
emailpath_tuples = find_emailpaths(all_emails)

In [15]:
# Count POI emails I have information for 
all_poi = set([tup[1] for tup in emailpath_tuples if tup[3]])
print 'Total poi emails: ', len(all_poi)
print '% of total: ', round(float(len(all_poi))/len(all_emails)*100,1)

Total poi emails:  18
% of total:  9.8


### Extract message information from email archive
Use the `emailpath_tuples` list generated above to extract useful word information from the email archive. Each email will be associated with cleaned up text of the messages it sent and received

In [17]:
from nltk.stem.snowball import SnowballStemmer

def parseOutText(email_path):
    """ 
    Extracts text and sender, receiver data from email found in a given path
    
    Return a dict with:
        -sender
        -receivers: [list]
        -original
        -stemmed (original with all words stemmed)
        
    """
    try:
        # File with individual email opened and extracted
        with open(email_path, 'r') as f:
            full_text = f.read()
    except IOError:
        print 'ERRROR: ', email_path
        return None
    
    # Get metadata
    end_meta = re.compile(r'X-FileName:.*')
    meta_s = end_meta.search(full_text)
    idx = meta_s.end()
    meta = full_text[:idx]
    
    meta = re.sub(r'\s', ' ', meta)
    
    # Get to and from emails from metadata
    def email_extract(raw_string, searched_string):
        try:
            result = re.search(raw_string, searched_string)
            emails = result.group(1)
            emails = emails.strip()
            email_list = emails.split(',')
            email_list = [word.strip() for word in email_list if word != '']
        except AttributeError:
            print 'SEND/RECEIVE ERROR', email_path
            return None
        return email_list
    
    from_emails = email_extract(r'From: (.*) To:', meta)
    to_emails = email_extract(r'To: (.*) Subject:', meta)
    
    # get content
    content = full_text[idx+1:]
    # remove reply and forward emails
    original = content.split('-----Original Message-----')[0]

    original_alpha = re.sub(r'\W', ' ', original)    # remove non-alpahnumerics
    original_alpha = re.sub(r'\s', ' ', original_alpha) #remove whitespace
    original_letter = re.sub(r'\d', '', original_alpha)
    # stem the words
    original_list = original_letter.split()
    words = ''
    stemmer = SnowballStemmer("english")
    for word in original_list:
        stemmed_word = stemmer.stem(word)
        words = words + stemmed_word + " " 
    dicto = {
            'sender': from_emails,
            'receivers': to_emails,
            'original': original_alpha,
            'stemmed': words
            }
    
    return dicto

In [18]:
# Some emails don't follow the regex rules for senders receivers. Generate an error but
# otherwise not worry
t = parseOutText('/Users/TRN/Desktop/raw_data/ml/maildir/martin-t/sent_items/188.')

SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/martin-t/sent_items/188.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/martin-t/sent_items/188.


In [19]:
# Example dict when the sender receivers didn't work out
t

{'original': '  You guys keep sending me this message and I keep telling you that I have never used this system   Please call me and tell what you issue is at ext  33079 because I have never logged into this system before   Tom   ',
 'receivers': None,
 'sender': None,
 'stemmed': u'you guy keep send me this messag and i keep tell you that i have never use this system pleas call me and tell what you issu is at ext becaus i have never log into this system befor tom '}

In [20]:
def word_dict_maker(emailpath_tuples, temp_save=False):
    """
    Makes word_dict from all of the emails specified in email path

    Input:
    List of tuples with:
    tup[0] = path to file with paths organized by email address
    tup[1] = email address
    tup[2] = whether emails were to or from email address
    tup[3] = True if email belongs to POI False otherwise

    output:
    word_dict = {
                 'email' : {'poi': True/False
                            'to (or from)': ['sender': from_email,
                                             'receiver': [to_emails],
                                             'original': original_alpha,
                                             'stemmed': words ]
                            }
                 }
    """
    # Assumes maildir directory is in current user Desktop rather than project
    # worspace
    
    maildir_path = DATA_PATH

    word_dict = {}
    count = 0
    for tup in emailpath_tuples:
        print tup[1], '\n**************\n'

        # Main dict entry for the email address being considered
        if tup[1] not in word_dict:
            word_dict[tup[1]] = {}
            word_dict[tup[1]]['poi'] = tup[3] 
            
        #print word_dict

        # file with paths to emails to/from adress opened
        with open(tup[0], 'r') as email_paths:
            dicto_list = []
            for email_path in email_paths:
                #email_path slice removes a top directory that was removed and newline
                email_path = os.path.join(maildir_path, email_path[20:-1])
                #print email_path
                dicto = parseOutText(email_path)
                dicto_list.append(dicto)
                print '.',
                
        if tup[2] == 'from':
            print 'FROM'
            word_dict[tup[1]]['from'] = dicto_list
        elif tup[2] == 'to':
            print 'TO'
            word_dict[tup[1]]['to'] = dicto_list

        #save to file every 100 loops if temp_save=True
        if temp_save:
            count += 1
            if count%100 == 0:
                with open('temp.pkl', 'w') as f:
                    pickle.dump(word_dict, f)
            elif count == len(emailpath_tuples):
                with open('word_dict_subset.pkl', 'w') as f:
                    pickle.dump(word_dict, f)

    return word_dict

In [21]:
 new_dict = word_dict_maker(emailpath_tuples)

a..martin@enron.com 
**************

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/martin-t/sent_items/188.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/martin-t/sent_items/188.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . FROM
adam.umanoff@enron.com 
**************

. . . . . . . . . . . . . . . . . . FROM
ben.glisan@enron.com 
**************

. . . . . . . . . . . . . . . . FROM
bill.cordes@enron.com 
**************

. . . . . . . . . . . . FROM
brian.redmond@enron.com 
**************

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/whalley-l/all_documents/2.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/whalley-l/all_documents/2.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/whalley-l/contacts/2.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/whalley-l/contacts/2.
. . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/whalley-l/discussion_threads/2.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/whalley-l/discussion_threads/2.
. . . . . . . . . . . . . . . . . . . . 

. SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/68.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/68.
. SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/69.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/69.
. SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/7.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/7.
. SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/70.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/70.
. SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/71.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/71.
. SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/72.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/derrick-j/contacts/72.
. SEND/RECEIVE ERR

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . FROM
james.hughes@enron.com 
**************

. . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/all_documents/404.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/all_documents/404.
. . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/archiving/untitled/1922.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/archiving/untitled/1922.
. . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/calendar/untitled/404.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/calendar/untitled/404.
. . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/discussion_threads/363.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/discussion_threads/363.
. . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kean-s/personnel/13.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildi

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

...

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kaminski-v/all_documents/10065.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kaminski-v/all_documents/10065.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kaminski-v/all_documents/10490.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kaminski-v/all_documents/10490.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kaminski-v/weather/11.
SEND/RECEIVE ERROR /Users/TRN/Desktop/raw_data/ml/maildir/kaminski-v/weather/11.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [22]:
full_text_path = os.path.join(DATA_PATH, 'full_text_dict.pkl')
with open(full_text_path, 'wb') as f:
    pickle.dump(new_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
full_text_path = os.path.join(DATA_PATH, 'full_text_dict.pkl')
with open(full_text_path, 'rb') as f:
    new_dict = pickle.load(f)

### Tf-Idf feature extraction

With the data organized into a dictionary extract text features based on term-frequency times inverse document-frequency (Tf-Idf) vectorization

In [4]:
def email_list_and_labels(word_dict):
    """
    Consolidates all the emails into a single string that can be used for 
    TfIdf vectorization
    
    Returns a list of dictos:
    {email: email
    poi: True/False
    to_text: all words in emails to this email
    from_text: all words in emails from this emails}
    """
    dicto_list = []
    
    for key in word_dict:
        
        dicto = {'email': key, 'poi': word_dict[key]['poi']}
        
        to_compilation = []
        from_compilation = []
        
        if 'to' in word_dict[key]:
            for temp_d in word_dict[key]['to']:
                to_compilation.append(temp_d['stemmed'])
                
        if 'from' in word_dict[key]:
            for temp_d in word_dict[key]['from']:
                from_compilation.append(temp_d['stemmed'])
        
        to_compilation = ''.join(to_compilation)
        from_compilation = ''.join(from_compilation)
        
        dicto['to_text'] = to_compilation
        dicto['from_text'] = from_compilation
        
        dicto_list.append(dicto)
        
        print '.',

    return dicto_list

In [5]:
text_dicts = email_list_and_labels(new_dict)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


Separate emails into to, from, or all categories

In [6]:
labels_emails = [(dicto['poi'], 
                  dicto['to_text'], 
                  dicto['from_text'], 
                  dicto['to_text'] + ' ' + dicto['from_text'],
                  dicto['email'])
                 for dicto in text_dicts]

In [7]:
labels, to_text, from_text, all_text, emails = zip(*labels_emails)

In [8]:
print(len(to_text), len(from_text), len(all_text), len(emails))

(90, 90, 90, 90)


Perform the actual vectorization

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')

from_array = vectorizer.fit_transform(from_text)
to_array = vectorizer.fit_transform(to_text)
all_array = vectorizer.fit_transform(all_text)
print('From array shape: {}, To array shape: {}, All array shape: {}' 
      .format(from_array.shape, to_array.shape, all_array.shape))

From array shape: (90, 42115), To array shape: (90, 62495), All array shape: (90, 74006)


In [9]:
from sklearn.model_selection import train_test_split

train_from_features, test_from_features, train_from_labels, test_from_labels =\
train_test_split(from_array, labels, test_size=0.4, random_state=30)

train_to_features, test_to_features, train_to_labels, test_to_labels =\
train_test_split(to_array, labels, test_size=0.4, random_state=30)

train_all_features, test_all_features, train_all_labels, test_all_labels =\
train_test_split(all_array, labels, test_size=0.4, random_state=30)

Test the word vectors using Naive Bayes. The testing compares vectors using words from all messages to and from an email with those just from an email. It also compares different selection methods.

In [29]:
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, mutual_info_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

def tester(percent, train_features, test_features, train_labels, test_labels):
    scr_list = [f_classif, chi2] #, mutual_info_classif]
    clf = GaussianNB()
    for scr in scr_list:
        print(scr)
        selector = SelectPercentile(scr, percentile=percent)
        try:
            train_features_sel = selector.fit_transform(train_features, train_labels).toarray()
            test_features_sel = selector.transform(test_features).toarray()
        except AttributeError:
            train_features_sel = selector.fit_transform(train_features, train_labels)
            test_features_sel = selector.transform(test_features)
        clf.fit(train_features_sel, train_labels)
        pred = clf.predict(test_features_sel)
        print(classification_report(test_labels, pred))

In [41]:
print '10% FROM:'
tester(10, train_from_features, test_from_features, train_from_labels, test_from_labels)
print '#########'
print '10% TO:'
tester(10, train_to_features, test_to_features, train_to_labels, test_to_labels)
print '#########'
print '10% ALL:'
tester(10, train_all_features, test_all_features, train_all_labels, test_all_labels)
print '#########'
print '5% FROM:'
tester(5, train_from_features, test_from_features, train_from_labels, test_from_labels)
print '#########'
print '5% TO:'
tester(5, train_to_features, test_to_features, train_to_labels, test_to_labels)
print '#########'
print '5% ALL:'
tester(5, train_all_features, test_all_features, train_all_labels, test_all_labels)
print '#########'

10% FROM:
<function f_classif at 0x1122bc9b0>
             precision    recall  f1-score   support

      False       0.85      0.79      0.81        28
       True       0.40      0.50      0.44         8

avg / total       0.75      0.72      0.73        36

<function chi2 at 0x1122bc6e0>
             precision    recall  f1-score   support

      False       0.81      0.89      0.85        28
       True       0.40      0.25      0.31         8

avg / total       0.72      0.75      0.73        36

<function mutual_info_classif at 0x11251a500>
             precision    recall  f1-score   support

      False       0.81      0.79      0.80        28
       True       0.33      0.38      0.35         8

avg / total       0.71      0.69      0.70        36

#########
10% TO:
<function f_classif at 0x1122bc9b0>
             precision    recall  f1-score   support

      False       1.00      0.21      0.35        28
       True       0.27      1.00      0.42         8

avg / total      

From words combined by selection using f_classif yields the f1 score for the True classification: 0.44 (the next best is also 0.44 but it relies on all words -which are 2x larger and the mutual-information selection, which is more time consuming)

### Save  a `vector_data` dictionary with the new text features

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')

In [13]:
from_array = vectorizer.fit_transform(from_text)
selector = SelectPercentile(f_classif, percentile=10)
sel_from_array = selector.fit_transform(from_array, labels)
sel_idxs = selector.get_support(indices=True)

vocab = vectorizer.vocabulary_ #list of word used in the vector. Will serve as keys
print 'Vocab dictionary before selection: ', len(vocab),\
      'compare to: ', from_array.shape[1] 
sel_vocab = {key: item for key, item in vocab.items() if item in sel_idxs}
print 'Vocab dictionary AFTER selection: ', len(sel_vocab),\
      'compare to: ', sel_from_array.shape[1]

Vocab dictionary before selection:  42115 compare to:  42115
Vocab dictionary AFTER selection:  4212 compare to:  4212


In [37]:
vocab_list = [[key, idx] for key, idx in vocab.items()]
keys, idxs = zip(*vocab_list)
keys, idxs = np.array(keys), np.array(idxs)

In [39]:
keys = keys[np.argsort(idxs)]

In [50]:
sel_vocab_list = keys[selector.get_support()]
print len(sel_vocab_list), 'compare with:', sel_from_array.shape[1]

4212 compare with: 4212


In [54]:
sel_vocab_list[100]

u'alcan'

In [16]:
with open('data/final_project_dataset.pkl', 'r') as data_file:
    data_dict = pickle.load(data_file)

del data_dict['TOTAL']
del data_dict['LOCKHART EUGENE E']

email_to_names =\
[(key, item['email_address']) for key, item in data_dict.items()]

In [17]:
print(len(email_to_names), email_to_names[0])

(144, ('METTS MARK', 'mark.metts@enron.com'))


In [18]:
# Out of those 144 names, how many are in the from_array?
in_list = [tup[1] for tup in email_to_names if tup[1] in emails]
print len(in_list)

86


In [74]:
# There are 4 emails missing, find out what they are
out_list = [email for email in emails if email not in in_list]
print(out_list)

['m..forney@enron.com', 'tim.despain@enron.com', 'larry.lawyer@enron.com', 'jeff.richter@enron.com']


In [75]:
# See if missing names are pois

# emailpath_tuples tup indices:
# tup[0] = path to file with paths organized by email address
# tup[1] = email address
# tup[2] = whether emails were to or from email address
# tup[3] = True if email belongs to POI False otherwise

for tup in emailpath_tuples:
    if tup[1] in out_list:
        print(tup[3])

True
True
True
True
True
True
True
True


All values not in original dataset are POIs, will include in new dictionary.

In [19]:
vector_only_names_emails = (('FORNEY M', 'm..forney@enron.com'),
                            ('DESPAIN TIM', 'tim.despain@enron.com'),
                            ('LAWYER LARRY', 'larry.lawyer@enron.com'),
                            ('RICHTER JEFF', 'jeff.richter@enron.com'))
print(len(email_to_names))
email_to_names.extend(vector_only_names_emails)
print(len(email_to_names))

144
148


At this point I need to add the emails not currently in the `vector_dict` but in the original `data_dict` and impute some value to them.

In [20]:
data_only_names_emails = [tup for tup in email_to_names if tup[1] not in emails]
print len(data_only_names_emails), 'compare to: ', len(data_dict) - len(in_list)

58 compare to:  58


In [21]:
data_names, data_emails =zip(*data_only_names_emails)
all_emails = list(emails) + list(data_emails)
print len(all_emails), 'compare to: ', len(email_to_names)

148 compare to:  148


In [22]:
data_only_labels = [data_dict[name]['poi'] for name in data_names]
all_labels = list(labels) + data_only_labels
print len(all_labels), 'compare to:', len(all_emails)

148 compare to: 148


In [58]:
import numpy as np

to_append = np.zeros((len(data_emails), sel_from_array.shape[1]))
to_append.fill(np.nan)
print sel_from_array.shape, to_append.shape

(90, 4212) (58, 4212)


In [59]:
new_from_array = np.concatenate((sel_from_array.toarray(), to_append), axis=0)
new_from_array.shape

(148, 4212)

In [96]:
from sklearn.preprocessing import Imputer

imp = Imputer(strategy='most_frequent')
imp_new_from_array = imp.fit_transform(new_from_array)
print imp_new_from_array.shape

(148, 4212)


In [97]:
from sklearn.model_selection import train_test_split

rdm = 200

train_features_sel, test_features_sel, train_labels_sel, test_labels_sel =\
train_test_split(sel_from_array.toarray(), labels, test_size=0.4, random_state=rdm)

train_features, test_features, train_labels, test_labels =\
train_test_split(imp_new_from_array, all_labels, test_size=0.4, random_state=rdm)

from sklearn.naive_bayes import GaussianNB

print 'Original: '
clf = GaussianNB()
clf.fit(train_features_sel, train_labels_sel)
pred = clf.predict(test_features_sel)
print(classification_report(test_labels_sel, pred))

print 'Imputed with additions: '
clf = GaussianNB()
clf.fit(train_features, train_labels)
pred = clf.predict(test_features)
print(classification_report(test_labels, pred))

Original: 
             precision    recall  f1-score   support

      False       0.92      1.00      0.96        33
       True       0.00      0.00      0.00         3

avg / total       0.84      0.92      0.88        36

Imputed with additions: 
             precision    recall  f1-score   support

      False       0.90      1.00      0.95        53
       True       1.00      0.14      0.25         7

avg / total       0.91      0.90      0.87        60



In [98]:
# Create the actual dictionary
vector_dict = {}
for ie, email in enumerate(all_emails):
    for tup in email_to_names:
        if tup[1] == email:
            name = tup[0]
            vector_dict[name]  = {}
            vector_dict[name]['poi'] = all_labels[ie]
            for iw, word in enumerate(sel_vocab_list):
                vector_dict[name][word] = float(imp_new_from_array[ie,iw])

In [99]:
for key, item in vector_dict.items():
    if len(item) != 4213:
        print key

In [100]:
vector_dict['METTS MARK'].keys()[:20]

[u'chippawa',
 u'mccaffrey',
 u'gentelmen',
 u'perrino',
 u'towncar',
 u'asian',
 u'galvez',
 u'sci',
 u'hate',
 u'tieback',
 u'mws',
 u'ljminvest',
 u'pedro',
 u'aug',
 u'enxco',
 u'deviat',
 u'semipro',
 u'reorganis',
 u'hedg',
 u'mwh']

In [102]:
import pickle
import os

HOME_PATH = os.path.expanduser('~')
DATA_PATH = os.path.join(HOME_PATH, 'Desktop', 'raw_data', 'ml')
vector_path = os.path.join(DATA_PATH, 'vector_dict.pkl')
name_path = os.path.join(DATA_PATH, 'vector_names.pkl')

with open(vector_path, 'wb') as f:
    pickle.dump(vector_dict, f)

with open(vector_path, 'wb') as f:
    pickle.dump(sel_vocab_list, f)

In [101]:
def featureFormat(dictionary, features, remove_NaN=True,
                  remove_all_zeroes=True, remove_any_zeroes=False,
                  sort_keys=False, keep_keys=False):
    """ convert dictionary to numpy array of features
        remove_NaN = True will convert "NaN" string to 0.0
        remove_all_zeroes = True will omit any data points for which
            all the features you seek are 0.0
        remove_any_zeroes = True will omit any data points for which
            any of the features you seek are 0.0
        sort_keys = True sorts keys by alphabetical order. Setting the value as
            a string opens the corresponding pickle file with a preset key
            order (this is used for Python 3 compatibility, and sort_keys
            should be left as False for the course mini-projects).
        NOTE: first feature is assumed to be 'poi' and is not checked for
            removal for zero or missing values.
    """
    return_list = []
    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()
        print 'len keys: ', len(keys)

    for key in keys:
        tmp_list = []
        if keep_keys:
            tmp_list.append(key)   
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print "error: key ", feature, " not present"
                return
            value = dictionary[key][feature]
            if value == "NaN" and remove_NaN:
                value = 0
            elif value == "NaN":
                value = np.nan
            tmp_list.append(float(value))

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)

In [103]:
from tools.feature_format import featureFormat

print len(vector_dict), len(sel_vocab_list)
data = featureFormat(vector_dict, sel_vocab_list, remove_NaN=True)
labels, features = data[:,0], data[:,1:]
print data.shape, labels.shape, features.shape

148 4212
(90, 4212) (90,) (90, 4211)
